In [25]:
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, BatchNormalization, MaxPool2D,Dropout
from data_utils import *
from keras.datasets import cifar10

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))###the configuration of Keras

In [26]:
def complex_cnn(classes, input_shape):
    """
    * 7x7 layer Conv layer with 32 filters
    * BN Layer
    * ReLu Activation Layer
    * 2x2 Max Polling Layer
    * 7x7 layer Conv layer with 64 filters
    * BN Layer
    * ReLu Activation Layer
    * 2x2 Max Polling Layer
    * Dense Layer with 1024 output units.
    * BN Layer
    * ReLU
    * Dense Layer with 10 outputs.
    """
    complex_model = Sequential()
    complex_model.add(Conv2D(filters=32,
                             kernel_size=7,
                             padding='same',
                             input_shape=input_shape,
                             ))
    complex_model.add(Dropout(rate=0.2))
    complex_model.add(BatchNormalization())
    complex_model.add(Activation('relu'))
    complex_model.add(MaxPool2D(padding='same'))
    #cov1 and pooling1
    complex_model.add(Conv2D(filters=64,
                             kernel_size=7,
                             padding='same',
                             input_shape=input_shape,
                             ))
    complex_model.add(Dropout(rate=0.2))
    complex_model.add(BatchNormalization())
    complex_model.add(Activation('relu'))
    complex_model.add(MaxPool2D(padding='same'))
    
    complex_model.add(Flatten())
    complex_model.add(Dense(1024))
    complex_model.add(BatchNormalization())
    complex_model.add(Activation('relu'))
    complex_model.add(Dense(classes))
    complex_model.add(Activation('softmax'))
    return complex_model

In [27]:
class CIFAR10Data(object):
    def __init__(self):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = cifar10.load_data()
        self.classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        print('CIFAR10 Training data shape:', self.x_train.shape)
        print('CIFAR10 Training label shape', self.y_train.shape)
        print('CIFAR10 Test data shape', self.x_test.shape)
        print('CIFAR10 Test label shape', self.y_test.shape)

    def get_stretch_data(self, subtract_mean=True):
        """
        reshape X each image to row vector, and transform Y to one_hot label.
        :param subtract_mean:Indicate whether subtract mean image.
        :return: x_train, one_hot_y_train, x_test, one_hot_y_test
        """
        num_classes = len(self.classes)
        x_train = np.reshape(self.x_train, (self.x_train.shape[0], -1)).astype('float16')
        y_train = keras.utils.to_categorical(self.y_train, num_classes)

        x_test = np.reshape(self.x_test, (self.x_test.shape[0], -1)).astype('float16')
        y_test = keras.utils.to_categorical(self.y_test, num_classes)

        if subtract_mean:
            mean_image = np.mean(x_train, axis=0).astype('uint8')
            x_train -= mean_image
            x_test -= mean_image
            # print(x_mean[:10])
            # plt.figure(figsize=(4, 4))
            # plt.imshow(x_mean.reshape((32, 32, 3)))
            # plt.show()

        return x_train, y_train, x_test, y_test

    def get_data(self, subtract_mean=True, output_shape=None):

        num_classes = len(self.classes)
        x_train = self.x_train
        x_test = self.x_test
        x_train = x_train.astype('float')
        y_train = keras.utils.to_categorical(self.y_train, num_classes)

        x_test = x_test.astype('float')
        y_test = keras.utils.to_categorical(self.y_test, num_classes)

        if subtract_mean:
            mean_image = np.mean(x_train, axis=0)
            x_train -= mean_image
            x_test -= mean_image
        return x_train, y_train, x_test, y_test
    
from keras.datasets import cifar10
cifar10_data = CIFAR10Data()
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

('CIFAR10 Training data shape:', (50000, 32, 32, 3))
('CIFAR10 Training label shape', (50000, 1))
('CIFAR10 Test data shape', (10000, 32, 32, 3))
('CIFAR10 Test label shape', (10000, 1))


In [28]:
print x_train.shape

(50000, 32, 32, 3)


In [29]:
complex_model = complex_cnn(classes=10, input_shape=x_train.shape[1:])
complex_model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-3),
                      loss=keras.losses.categorical_crossentropy,
                      metrics=['accuracy'],
                      )
complex_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        4736      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_9 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)        100416    
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 64)        0         
__________

In [51]:
##49000 for train 1000 for validation
history = complex_model.fit(x_train, y_train, batch_size=64, epochs=5, validation_split=0.02)


Train on 49000 samples, validate on 1000 samples
Epoch 1/5
49000/49000 [==============================] - 249s 5ms/step - loss: 1.3773 - acc: 0.5235 - val_loss: 1.3106 - val_acc: 0.5610
Epoch 2/5
49000/49000 [==============================] - 462s 9ms/step - loss: 0.9589 - acc: 0.6653 - val_loss: 1.1927 - val_acc: 0.5730
Epoch 3/5
49000/49000 [==============================] - 248s 5ms/step - loss: 0.7606 - acc: 0.7317 - val_loss: 1.1043 - val_acc: 0.6330
Epoch 4/5
49000/49000 [==============================] - 248s 5ms/step - loss: 0.6161 - acc: 0.7842 - val_loss: 1.0923 - val_acc: 0.6410
Epoch 5/5
49000/49000 [==============================] - 250s 5ms/step - loss: 0.4841 - acc: 0.8303 - val_loss: 1.1950 - val_acc: 0.6330


In [52]:
loss, acc = complex_model.evaluate(x_test, y_test)
print('evaluate loss :%f acc:%f' % (loss, acc))

10000/10000 [==============================] - 12s 1ms/step
evaluate loss :1.200404 acc:0.633400


('PCA runtime:', 1.1489500999450684)


ValueError: Error when checking input: expected conv2d_5_input to have 4 dimensions, but got array with shape (50000, 40)